In [ ]:
###### TPKL_G01_168_22
import numpy as np
import pandas as pd
from matplotlib.pyplot import grid, figure, plot, savefig
import matplotlib.pyplot as plt
t0 = 0
lat = 42.46386370313852 #shitot ot ecvat   MARKS   TPLK
lon = 78.52905690246126# dolg merid-i po ekvat
info = []
l_data = []
arrData1=[]# matrix glavn SS(ws^2)
arrData2=[]# matrix glavn S(ws)
arrData3=[]# matrix glavn S(ws).T 
arrData4=[]# matrix glavn S(w)
arrData5=[]# matrix glavn S(wI)
arrData6=[]# matrix glavn SS(wsI)

#chi = 'G17'
for i in range(0, 6):
    if i==0:
        chi = 'G05'    
    if i==1:
        chi = 'G10'
    if i==2:
        chi = 'G13'  
    if i==3:
        chi = 'G15'
    if i==4:
        chi = 'G18'
    if i==5:
        chi = 'G23'
    #if i==7:
    #    chi = 'G30'    
        
    txt = "TPKL_" +chi + "_168_22.dat"
    txt2 = "TPKL_G"+chi +"_168_22.csv"
    file1 = open (txt)
    #file1 = open ("TPKL_G17_168_22.dat")
    allLines = file1.readlines()
    file1linecount = len(allLines)
    file1.seek(0) # peremestit ukozatel v nachal

    for ln in range(file1linecount) :#3700 strok все строки

        line  = file1.readline()
        if ln < 10 :
            param=line.strip() #str
            #param = param.split()
            info.append(param)

        if ln > 10 :
            param=line.strip() #str
            param = param.split()
            #print(param)
            #np.array(param)
            l_data.append(param)


    #array = np.array(arr)
    #array = array.astype(float)
    arr = np.array(l_data)
    arr = arr.astype(float)
    # Create the dataframe

    df = pd.DataFrame(arr, columns=['tsn', 'hour', 'el', 'az','phase','code','valid'])
    ######print(info)
    df['Cr']=  df['code']-df['phase'] 
    df['Coef']= ((1/df['Cr'].count())*df['Cr'].sum())
    df['sTEC']= df['phase']+df['Coef']
    df['vTEC'] =  df['sTEC']*  (np.cos  (np.arcsin(  (6400*np.cos(df['el']*3.14/180))  /  (6400+350) )  )     ) #
    #df['STEC']=df['sTEC']
    #df['VTEC']=df['vTEC']
    df.plot.scatter(x='hour',y='vTEC',c='DarkBlue')
    df.plot.scatter(x='hour',y='sTEC',c='DarkBlue')
# tochka prokola
    df['psi'] = np.pi/2 - np.radians(df['el'])- np.arcsin(6371 * np.cos( np.radians(df['el']) )/(6371+350) ) #1.209
    df['lat']=  np.degrees(np.arcsin( np.sin(np.radians(lat)) * np.cos(df['psi']) + np.cos(np.radians(lat))*np.sin(df['psi']) *np.cos(np.radians(df['az'])) ))
    df['lon'] = np.degrees(np.radians(lon)+ np.arcsin( np.sin( df['psi'] ) * np.sin(np.radians( df['az'] ) ) * (1/np.cos( np.radians( df['lat'] )) ) )  ) 
    #print(dd)
    #  INterval posled 10 
    dd = df.iloc[:10]
    #print(dd)
    arrt0= dd['hour'].iloc[[0]] # 
    arrtE= dd['hour'].iloc[[-1]] #
    #ST= dd[['sTEC','vTEC']].iloc[[-24]] # stec vtec в шт
    #arrST = ST.mean().values # сред по stec vtec штук в масив
    #print(dd['hour'])
    t0 =arrt0.values # t 0-e
    tE =arrtE.values # t 0-e
    print('t0 - tend ', t0, ' ', tE)

    Interval = tE-t0

    dd['t0'] = t0[0]#  23.8 каждый цикл 
##    dd['t0'] = 10.758333#  23.8 каждый цикл 
    dd['delt_t'] = dd['hour']-dd['t0']
##    dd['delt_t']=dd['hour']-dd['hour'].shift(1)
    
    #dd['delt_t'] = dd['hour']-dd['hour']
    #df['delt_t'] = df.hour - dd.hour.shift(1)
    #dd['S'] = np.degrees(np.cos(  np.arcsin( 6371* np.sin(0.97*(90-np.radians(df['el'])))/(6371+350) )    )**-1 )# factor naklona
    #dd['S'] = np.degrees(np.cos(  np.arcsin( 6371* np.cos(np.radians(df['el']))/(6371+350) )    )**-1) # factor naklona
    dd['S'] = np.cos(  np.arcsin( 6371* np.sin(0.97*np.radians(90-df['el']))/(6371+350) )    )**-1 # factor naklona
#    dd['S'] = np.cos(  np.arcsin( 6371* np.cos(np.radians(df['el']))/(6371+350) )    )**-1 # factor naklona
    #dd['S'] =np.degrees( np.cos(  np.arcsin( 6371* np.cos(np.radians(df['el']))/(6371+350) )    )**-1 )# factor naklona grad
    dd['w'] = 1/dd['S'] *(  1+ ((2* dd['delt_t'] )/Interval)**2     )**-1 # ves =  delt t / interval 
    dd['S^2'] = dd['S']**2
    #dd['STEC']=arrST[0] # sred
    #dd['VTEC']=arrST[1]
    #print(dd[['w','el']])

    # dd['delt_t'] t 
    dd['delt_t^2'] = dd['delt_t']**2
    dd['delt_t^3'] = dd['delt_t']**3
    dd['delt_t^4'] = dd['delt_t']**4
    # delt = tochka prokola - koord pribora 
    dd['delt_lat']= dd['lat']-lat
    dd['delt_lat^2']= (dd['lat']-lat)**2
    dd['delt_lat^3']= (dd['lat']-lat)**3
    dd['delt_lat^4']= (dd['lat']-lat)**4
    dd['delt_lon']= dd['lon']-lon
    dd['delt_lon^2']= (dd['lon']-lon)**2
    dd['delt_lon^3']= (dd['lon']-lon)**3
    dd['delt_lon^4']= (dd['lon']-lon)**4

    dd['w*S^2'] =                 dd['w']*dd['S^2']
    dd['w*S^2*d_lat'] =           dd['w']*dd['S^2']*dd['delt_lat']
    dd['w*S^2*d_lat^2'] =         dd['w']*dd['S^2']*dd['delt_lat^2']
    dd['w*S^2*d_lat^3'] =         dd['w']*dd['S^2']*dd['delt_lat^3']
    dd['w*S^2*d_lat^4'] =         dd['w']*dd['S^2']*dd['delt_lat^4']
    dd['w*S^2*d_lon'] =           dd['w']*dd['S^2']*dd['delt_lon']
    dd['w*S^2*d_lon^2'] =         dd['w']*dd['S^2']*dd['delt_lon^2']
    dd['w*S^2*d_lon^3'] =         dd['w']*dd['S^2']*dd['delt_lon^3']
    dd['w*S^2*d_lon^4'] =         dd['w']*dd['S^2']*dd['delt_lon^4']
    dd['w*S^2*d_t'] =             dd['w']*dd['S^2']*dd['delt_t']
    dd['w*S^2*d_t^2'] =           dd['w']*dd['S^2']*dd['delt_t^2']
    dd['w*S^2*d_t^3'] =           dd['w']*dd['S^2']*dd['delt_t^3']
    dd['w*S^2*d_t^4'] =           dd['w']*dd['S^2']*dd['delt_t^4']
    dd['w*S^2*d_lat*d_lon']=      dd['w']*dd['S^2']*dd['delt_lat']*dd['delt_lon']
    dd['w*S^2*d_lat*d_lon^2']=    dd['w']*dd['S^2']*dd['delt_lat']*dd['delt_lon^2']
    dd['w*S^2*d_lat^2*d_lon'] =   dd['w']*dd['S^2']*dd['delt_lat^2']*dd['delt_lon']
    dd['w*S^2*d_lat^2*d_lon^2'] = dd['w']*dd['S^2']*dd['delt_lat^2']*dd['delt_lon^2']
    dd['w*S^2*d_lat*d_t']=        dd['w']*dd['S^2']*dd['delt_lat']*dd['delt_t']
    dd['w*S^2*d_lat*d_t^2']=      dd['w']*dd['S^2']*dd['delt_lat']*dd['delt_t^2']
    dd['w*S^2*d_lat^2*d_t']=      dd['w']*dd['S^2']*dd['delt_lat^2']*dd['delt_t']
    dd['w*S^2*d_lat^2*d_t^2']=    dd['w']*dd['S^2']*dd['delt_lat^2']*dd['delt_t^2']
    dd['w*S^2*d_lon*d_t']=        dd['w']*dd['S^2']*dd['delt_lon']*dd['delt_t']
    dd['w*S^2*d_lon*d_t^2']=      dd['w']*dd['S^2']*dd['delt_lon']*dd['delt_t^2']
    dd['w*S^2*d_lon^2*d_t']=      dd['w']*dd['S^2']*dd['delt_lon^2']*dd['delt_t']
    dd['w*S^2*d_lon^2*d_t^2']=    dd['w']*dd['S^2']*dd['delt_lon^2']*dd['delt_t^2']

    dd['w*S'] =         dd['w']*dd['S']
    dd['w*S*d_lat'] =   dd['w']*dd['S']*dd['delt_lat']
    dd['w*S*d_lat^2'] = dd['w']*dd['S']*dd['delt_lat^2']
    dd['w*S*d_lon'] =   dd['w']*dd['S']*dd['delt_lon']
    dd['w*S*d_lon^2'] = dd['w']*dd['S']*dd['delt_lon^2']
    dd['w*S*d_t'] =     dd['w']*dd['S']*dd['delt_t']
    dd['w*S*d_t^2'] =   dd['w']*dd['S']*dd['delt_t^2']

    dd['w*I']=            dd['w']*dd['sTEC'] 
    
    dd['w*S*I'] =         dd['w']*dd['S']*dd['sTEC']
    dd['w*S*d_lat*I'] =   dd['w']*dd['S']*dd['delt_lat']*dd['sTEC']
    dd['w*S*d_lat^2*I'] = dd['w']*dd['S']*dd['delt_lat^2']*dd['sTEC']
    dd['w*S*d_lon*I'] =   dd['w']*dd['S']*dd['delt_lon']*dd['sTEC']
    dd['w*S*d_lon^2*I'] = dd['w']*dd['S']*dd['delt_lon^2']*dd['sTEC']
    dd['w*S*d_t*I'] =     dd['w']*dd['S']*dd['delt_t']*dd['sTEC']
    dd['w*S*d_t^2*I'] =   dd['w']*dd['S']*dd['delt_t^2']*dd['sTEC']
    #print(dd['sTEC'])
    dd.to_csv('T'+txt2,sep=';') 
    ###############SUM i-x izmereniy  ######################
    dd['(w*S^2)'] =          dd['w*S^2'].sum()
    dd['(w*S^2*d_lat)'] =    dd['w*S^2*d_lat'].sum()
    dd['(w*S^2*d_lat^2)'] =  dd['w*S^2*d_lat^2'].sum()
    dd['(w*S^2*d_lat^3)'] =  dd['w*S^2*d_lat^3'].sum()
    dd['(w*S^2*d_lat^4)'] =  dd['w*S^2*d_lat^4'].sum()
    dd['(w*S^2*d_lon)'] =    dd['w*S^2*d_lon'].sum()
    dd['(w*S^2*d_lon^2)'] =  dd['w*S^2*d_lon^2'].sum()
    dd['(w*S^2*d_lon^3)'] =  dd['w*S^2*d_lon^3'].sum()
    dd['(w*S^2*d_lon^4)'] =  dd['w*S^2*d_lon^4'].sum()
    dd['(w*S^2*d_t)'] =      dd['w*S^2*d_t'].sum()
    dd['(w*S^2*d_t^2)'] =    dd['w*S^2*d_t^2'].sum()
    dd['(w*S^2*d_t^3)'] =    dd['w*S^2*d_t^3'].sum()
    dd['(w*S^2*d_t^4)'] =    dd['w*S^2*d_t^4'].sum()    

    ###################

    dd['(w*S^2*d_lat*d_lon)']=      dd['w*S^2*d_lat*d_lon'].sum()
    dd['(w*S^2*d_lat*d_lon^2)']=    dd['w*S^2*d_lat*d_lon^2'].sum()
    dd['(w*S^2*d_lat^2*d_lon)'] =   dd['w*S^2*d_lat^2*d_lon'].sum()
    dd['(w*S^2*d_lat^2*d_lon^2)'] = dd['w*S^2*d_lat^2*d_lon^2'].sum()
    dd['(w*S^2*d_lat*d_t)']=        dd['w*S^2*d_lat*d_t'].sum()
    dd['(w*S^2*d_lat*d_t^2)']=      dd['w*S^2*d_lat*d_t^2'].sum()
    dd['(w*S^2*d_lat^2*d_t)']=      dd['w*S^2*d_lat^2*d_t'].sum()
    dd['(w*S^2*d_lat^2*d_t^2)']=    dd['w*S^2*d_lat^2*d_t^2'].sum()
    dd['(w*S^2*d_lon*d_t)']=        dd['w*S^2*d_lon*d_t'].sum()
    dd['(w*S^2*d_lon*d_t^2)']=      dd['w*S^2*d_lon*d_t^2'].sum()
    dd['(w*S^2*d_lon^2*d_t)']=      dd['w*S^2*d_lon^2*d_t'].sum()
    dd['(w*S^2*d_lon^2*d_t^2)']=    dd['w*S^2*d_lon^2*d_t^2'].sum()
    ############################
   
    dd['(w)'] =   dd['w'].sum()
    dd['(w*I)'] = dd['w*I'].sum()
    dd['(w*S)'] = dd['w*S'].sum()
    dd['(w*S*d_lat)'] =   dd['w*S*d_lat'].sum()
    dd['(w*S*d_lat^2)'] = dd['w*S*d_lat^2'].sum()
    dd['(w*S*d_lon)'] =   dd['w*S*d_lon'].sum()
    dd['(w*S*d_lon^2)'] = dd['w*S*d_lon^2'].sum()
    dd['(w*S*d_t)'] =     dd['w*S*d_t'].sum()
    dd['(w*S*d_t^2)'] =   dd['w*S*d_t^2'].sum()
    
    dd['(w*S*I)'] =       dd['w*S*I'].sum()                 
    dd['(w*S*d_lat*I)'] = dd['w*S*d_lat*I'].sum()
    dd['(w*S*d_lat^2*I)']=dd['w*S*d_lat^2*I'].sum()
    dd['(w*S*d_lon*I)'] = dd['w*S*d_lon*I'].sum()
    dd['(w*S*d_lon^2*I)']=dd['w*S*d_lon^2*I'].sum()
    dd['(w*S*d_t*I)'] =   dd['w*S*d_t*I'].sum()
    dd['(w*S*d_t^2*I)'] = dd['w*S*d_t^2*I'].sum()
#df['DataFrame Column'] = df['DataFrame Column'].astype(int)
    #print(dd['S'])

    dd.drop(['tsn','sTEC','vTEC','el', 'az','phase','code','valid','Cr','t0','w*S*d_lat^2','w*S*d_lon','w*S*d_t','w*S*d_t^2','w*S*d_lon^2','Coef','w*S*d_lat','psi','lat','lon',"w"], axis=1, inplace=True)    
    dd.drop(['w*S','S','S^2','delt_lat','delt_lon','delt_lat^2','delt_lon^2','w*S^2','w*S^2*d_lat','w*S^2*d_lat^2','w*S^2*d_lon','w*S^2*d_lon^2','w*S^2*d_t','w*S^2*d_t^2','delt_t','delt_t^2'], axis=1, inplace=True)
    dd.drop(['delt_t^3','delt_t^4','delt_lat^3','delt_lat^4','delt_lon^3','delt_lon^4','w*S^2*d_lat^3','w*S^2*d_lat^4','w*S^2*d_lon^3','w*S^2*d_lon^4','w*S^2*d_t^3','w*S^2*d_t^4'], axis=1, inplace=True)    
    dd.drop(['w*I','w*S*I','w*S*d_lat*I','w*S*d_lon*I','w*S*d_lon^2*I','w*S*d_t*I','w*S*d_t^2*I',], axis=1, inplace=True) 
    dd.drop(['w*S^2*d_lat*d_lon','w*S^2*d_lat*d_lon^2','w*S^2*d_lat^2*d_lon^2','w*S^2*d_lat^2*d_lon'], axis=1, inplace=True) 
    dd.drop(['w*S^2*d_lat*d_t','w*S^2*d_lat^2*d_t','w*S^2*d_lat*d_t^2','w*S^2*d_lat^2*d_t^2'], axis=1, inplace=True) 
    dd.drop(['w*S^2*d_lon*d_t','w*S^2*d_lon^2*d_t','w*S^2*d_lon*d_t^2','w*S^2*d_lon^2*d_t^2'], axis=1, inplace=True) 
    dd.drop(['w*S*d_lat^2*I'], axis=1, inplace=True)    

    ##print(dd.dtypes) float all
    #print(dd)
    dd.to_csv(txt2,sep=';') 
    arrData1.append(dd.values[0][1:26].tolist())
    arrData2.append(dd.values[0][28:35].tolist())
    arrData4.append(dd.values[0][26].tolist())
    arrData5.append(dd.values[0][27].tolist())
    arrData6.append(dd.values[0][35:].tolist())
    #print(dd['(w*S*d_lat*I)'])
    
    #print(type(dd.values[0][35:]))
#arrData6= list(map(int, arrData6))
#aD6 = np.array(arrData6)
#print((arrData6))
#arrData6 = np.array(arrData1)
#arrData6 = np.sum(arrData6[:, 1])
arrData1 = np.array(arrData1)
arrData1 = np.sum(arrData1,axis = 0)
arrData2 = np.array(arrData2)
arrData3 = arrData2.T
arrData4 = np.array(arrData4)
arrData5 = np.array(arrData5)
arrData6 = np.array(arrData6)

arrData6 = np.sum(arrData6,axis = 0)
#print(arrData2)


matrix_A = np.array([
                    
                    [arrData1[0],   arrData1[1],  arrData1[2],  arrData1[5],  arrData1[6],  arrData1[9],  arrData1[10],arrData3[0][0],arrData3[0][1],arrData3[0][2],arrData3[0][3],arrData3[0][4],arrData3[0][5] ], 
                    [arrData1[1],   arrData1[2],  arrData1[3],  arrData1[13], arrData1[14], arrData1[17], arrData1[18],arrData3[1][0],arrData3[1][1],arrData3[1][2],arrData3[1][3],arrData3[1][4],arrData3[1][5] ],
                    [arrData1[2],   arrData1[3],  arrData1[4],  arrData1[15], arrData1[16], arrData1[19], arrData1[20],arrData3[2][0],arrData3[2][1],arrData3[2][2],arrData3[2][3],arrData3[2][4],arrData3[2][5] ], 
                    [arrData1[5],  arrData1[13],  arrData1[15], arrData1[6],  arrData1[7], arrData1[21], arrData1[22], arrData3[3][0],arrData3[3][1],arrData3[3][2],arrData3[3][3],arrData3[3][4],arrData3[3][5] ],
                    [arrData1[6],  arrData1[14],  arrData1[16], arrData1[7],  arrData1[8], arrData1[23], arrData1[24], arrData3[4][0],arrData3[4][1],arrData3[4][2],arrData3[4][3],arrData3[4][4],arrData3[4][5] ],
                    [arrData1[9],  arrData1[17],  arrData1[19], arrData1[21], arrData1[23], arrData1[10], arrData1[11],arrData3[5][0],arrData3[5][1],arrData3[5][2],arrData3[5][3],arrData3[5][4],arrData3[5][5] ],
                    [arrData1[10]  ,arrData1[18], arrData1[20], arrData1[22], arrData1[24], arrData1[11], arrData1[12],arrData3[6][0],arrData3[6][1],arrData3[6][2],arrData3[6][3],arrData3[6][4],arrData3[6][5] ],
                    [arrData2[0][0],arrData2[0][1],arrData2[0][2],arrData2[0][3],arrData2[0][4],arrData2[0][5],arrData2[0][6],         arrData4[0],0,0,0,0,0   ] ,
                    [arrData2[1][0],arrData2[1][1],arrData2[1][2],arrData2[1][3],arrData2[1][4],arrData2[1][5],arrData2[1][6], 0,      arrData4[1],0,0,0,0   ] ,
                    [arrData2[2][0],arrData2[2][1],arrData2[2][2],arrData2[2][3],arrData2[2][4],arrData2[2][5],arrData2[2][6], 0,0,    arrData4[2],0,0,0   ] ,
                    [arrData2[3][0],arrData2[3][1],arrData2[3][2],arrData2[3][3],arrData2[3][4],arrData2[3][5],arrData2[3][6], 0,0,0,  arrData4[3],0,0    ] ,
                    [arrData2[4][0],arrData2[4][1],arrData2[4][2],arrData2[4][3],arrData2[4][4],arrData2[4][5],arrData2[4][6],  0,0,0,0,arrData4[4],0    ] ,
                    [arrData2[5][0],arrData2[5][1],arrData2[5][2],arrData2[5][3],arrData2[5][4],arrData2[5][5],arrData2[5][6],  0,0,0,0,0,arrData4[5]    ] ,
                    #[arrData2[6][0],arrData2[6][1],arrData2[6][2],arrData2[6][3],arrData2[6][4],arrData2[6][5],arrData2[6][6], 0,0,0,0,0,0,arrData4[6],0   ] ,
                    #[arrData2[7][0],arrData2[7][1],arrData2[7][2],arrData2[7][3],arrData2[7][4],arrData2[7][5],arrData2[7][6], 0,0,0,0,0,0,0,arrData4[7]   ] ,
                    
])
                
matrix_b = np.array([
    [arrData6[0]],
    [arrData6[1]],
    [arrData6[2]],
    [arrData6[3]],
    [arrData6[4]],
    [arrData6[5]],
    [arrData6[6]],
    [arrData5[0]],
    [arrData5[1]],
    [arrData5[2]],
    [arrData5[3]],
    [arrData5[4]],
    [arrData5[5]],
    #[arrData5[6]],
   
    
])     

#print('\n')                  
#print(matrix_A)
#print(arrData4)
#ww = np.linalg.lstsq(matrix_A.T, matrix_b, rcond=None)[0]
#ww
#xx = pd.Series(ww) 
#xx

In [1156]:
matrixA = pd.read_csv ('a.csv',sep = ';')
matrixB = matrixA['b']
matrixA.drop(['b'], axis=1, inplace=True)

b = matrixB.values
A = matrixA.values

#res = np.linalg.lstsq(A, B)
#x = np.arange(0, 9)
#A = np.array([x, np.ones(9)])
#A = np.random.rand(3,9)

# linearly generated sequence
#y = [19, 20, 20.5, 21.5, 22, 23, 23, 25.5, 24]

# obtaining the parameters of regression line
#w = np.linalg.lstsq(A.T, y, rcond=None)[0]

w = np.linalg.lstsq(A.T, b, rcond=None)[0]
w
x = pd.Series(w) 
x


0       8.147002
1      -0.294752
2       0.001580
3       0.204914
4       0.005200
5       2.268554
6       0.772931
7     169.100782
8     180.634633
9     118.222461
10    120.628181
11    136.746967
12    183.185140
dtype: float64

array([[1, 2],
       [1, 2]])

In [ ]:


#timeInt = df['hour'].tail(1)-df['hour'].head(1)
#print('timeInter ',timeInt)
#cnt = df.count()
#print(df.count())
#df['w'] = 1/df['S']  *  (1 + ( 2*df['hour']-df['hour'] / (df['hour']-df['hour']) ) )**-1
#d = df[(df['tsn'] == 453)]
#d = df.loc[(df['tsn'] == 453)]
#tiBig =df.hour.head(1).values
#tiEnd = df.hour.tail(1).values
#dfn = df.values
#dfn[-1][1]-dfn[0][1] 
#for i in range(0,484):
#    print(2*(dfn[i+1][1] - dfn[i][1]))



#df['S'] = np.cos(  np.arcsin( 6371* np.cos(np.radians(df['el']))/(6371+350) )    )**-1
#df['S2'] = np.cos(  np.arcsin( 6371* np.sin(0.97*np.radians(90-df['el']))/(6371+350) )    )**-1

#df['w'] = 1/df['S'] *(  1+ (2*0.016/0.25)**2     )**-1


#dj.drop(['el', 'az','phase','code','valid','Cr','Coef','psi','lat','lon'], axis=1, inplace=True)
dj['sumS2'] = dj['S^2'].sum()
dj['W_S2^2'] = dj['w']*dj['S^2']
dj['W S2^2 d_lat'] = dj['w']*dj['S^2']*dj['delt_lat']
dj['W S2^2 d_lat^2'] = dj['w']*dj['S^2']*dj['delt_lat^2']
dj['W S2^2 d_lon'] = dj['w']*dj['S^2']*dj['delt_lon']
dj['W S2^2 d_lon^2'] = dj['w']*dj['S^2']*dj['delt_lon^2']
dj['W S2^2 d_t'] = dj['w']*dj['S^2']*dj['delt_t']
dj['W S2^2 d_t^2'] = dj['w']*dj['S^2']*dj['delt_t^2']
dj['W_S2^2_sum'] = dj['W_S2^2'].sum()
dj['W S2^2 d_lat'] = dj['W S2^2 d_lat'].sum()
dj['W S2^2 d_lat^2'] = dj['W S2^2 d_lat^2'].sum()
dj['W S2^2 d_lon'] = dj['W S2^2 d_lon'].sum()
dj['W S2^2 d_lon^2'] = dj['W S2^2 d_lon^2'].sum()
dj['W S2^2 d_t'] = dj['W S2^2 d_t'].sum()
dj['W S2^2 d_t^2'] = dj['W S2^2 d_t^2'].sum()
dj.drop(['el', 'az','phase','code','valid','Cr','Coef','psi','lat','lon'], axis=1, inplace=True)
#vvv = dj.values
#for i in range(0,len(vvv)):
#    print(sum)
#for index, row in dj.iterrows():
#    print(row['S'])
#    print('1')
#out = SVA.loc[:,["S","S^2","w","delt_lat","delt_lon","delt_lat^2","delt_lon^2"]]
dj.to_csv(txt2,sep=';') 

#print(9.00e-04)
#dj

In [572]:
plt.show()
plt.figure(figsize=(15,7))
plt.plot(dd.hour,dd.phase , 'm', lw=1)
plt.plot(dd.hour,dd.code , 'c', lw=1)
plt.plot(dd.hour,dd.TEC , 'y', lw=1)
#plt.plot(df['VTEC'], 'g', lw=1)
#plt.plot(df['VtecL2'], 'b', lw=1)
plt.title('Code and Phase')
plt.legend(['Code tec', 'Phase tec', 'const+Phase tec'])
plt.xlabel('hour')
plt.ylabel('TEC')

AttributeError: 'DataFrame' object has no attribute 'phase'

<Figure size 1080x504 with 0 Axes>

In [ ]:
 for idx, sta in enumerate(self.__stalst__):
    dx, dy, dr = xyr[idx] # X-Xgrid, Y-Ygrid, квадрат(X-Xgrid^2  + Y-Ygrid^2)
    Wx     = W[i]
    Wy     = W[i+1]
    A[i]   = [ Wx*j for j in [1e0, 0e0,  dx, dy, 0e0, dy] ]
    A[i+1] = [ Wy*j for j in [0e0, 1e0, 0e0, dx, dy, -dx] ]
    b[i]   = sta.ve * Wx
    b[i+1] = sta.vn * Wy
    i += 2
assert i == N, "[DEBUG] Failed to construct ls matrices"
return A, b